In [1]:
import numpy as np
import stan
import nest_asyncio
import asyncio

from services.data_service import load_site_data
from solvers.stan import _theta_reg_data, _prior_hyperparams, _gamma_reg_data

Importing MCMC Module: mcmc_sampling_original.py


In [2]:
# Load sites' data
(
    zbar_2017,
    gamma_vals,
    z_2017,
    forestArea_2017_ha,
    theta_vals,
    gamma_coe_mean,
    theta_coe_mean,
    gamma_coe_vcov,
    theta_coe_vcov,
    theta_data,
    gamma_data,
) = load_site_data(
    10,
    norm_fac=1e11,
)

Data successfully loaded from '/Users/patricioh/Projects/project-amazon/data/hmc/hmc_10SitesModel.csv'


In [3]:
with open('../stan_models/full_model/priors.stan') as f:
        model_code = f.read()

In [4]:
y_theta, X_theta, N_theta, K_theta, G_theta = _theta_reg_data(10, theta_data)
y_gamma, X_gamma, N_gamma, K_gamma, G_gamma = _gamma_reg_data(10, gamma_data)

model_data = dict(
    S=10,
    K_theta=K_theta,
    K_gamma=K_gamma,
    N_theta=N_theta,
    N_gamma=N_gamma,
    G_theta=G_theta,
    G_gamma=G_gamma,
    pa_2017=44.9736197781184,
    **_prior_hyperparams(y_theta, X_theta, "theta"),
    **_prior_hyperparams(y_gamma, X_gamma, "gamma"),
)

# Compiling model
sampler = stan.build(program_code=model_code, data=model_data, random_seed=1)
fit = sampler.fixed_param(num_samples=1000)

KeyError: 'log_cattleSlaughter_valuePerHa_2017'